# Лабораторная работа №5


_ПИН-212 Жан-Лин Никита_

In [1]:
# Импорт классов для масштабирования числовых признаков
from sklearn.preprocessing import  StandardScaler

# Импорт библиотеки NumPy для работы с массивами и матрицами
import numpy as np

# Импорт библиотеки pandas для работы с данными в табличном формате
import pandas as pd

# Импорт функций для разделения данных на обучающий и тестовый наборы и для поиска оптимальных параметров модели
from sklearn.model_selection import train_test_split, GridSearchCV

# Импорт класса для построения модели дерева решений
from sklearn.tree import DecisionTreeClassifier

# Импорт класса для построения модели случайного леса
from sklearn.ensemble import RandomForestClassifier

# Импорт функций для оценки производительности модели
from sklearn.metrics import accuracy_score, classification_report

In [3]:
data = pd.read_csv('smoking_driking_dataset_Ver03.csv', encoding='utf-8')
print(data.head(5))

    sex  age  height  weight  waistline  sight_left  sight_right  hear_left  \
0  Male   35     170      75       90.0         1.0          1.0        1.0   
1  Male   30     180      80       89.0         0.9          1.2        1.0   
2  Male   40     165      75       91.0         1.2          1.5        1.0   
3  Male   50     175      80       91.0         1.5          1.2        1.0   
4  Male   50     165      60       80.0         1.0          1.2        1.0   

   hear_right    SBP  ...  LDL_chole  triglyceride  hemoglobin  urine_protein  \
0         1.0  120.0  ...      126.0          92.0        17.1            1.0   
1         1.0  130.0  ...      148.0         121.0        15.8            1.0   
2         1.0  120.0  ...       74.0         104.0        15.8            1.0   
3         1.0  145.0  ...      104.0         106.0        17.6            1.0   
4         1.0  138.0  ...      117.0         104.0        13.8            1.0   

   serum_creatinine  SGOT_AST  SGOT_AL

In [5]:
# Преобразуем 'sex' в числовую категорию
data['sex'] = data['sex'].map({'Male': 1, 'Female': 0}).astype(int)

# Преобразуем 'DRK_YN' в бинарные метки
data['DRK_YN'] = data['DRK_YN'].map({'Y': 1, 'N': 0}).astype(int)

# Проверка уникальных значений в целевой переменной
print("Уникальные значения в data['DRK_YN']:", data['DRK_YN'].unique())

Уникальные значения в data['DRK_YN']: [1 0]


In [7]:
columns_to_drop = [
    'SMK_stat_type_cd',  # Если не нужен
    'sight_left', 'sight_right',  # Если зрение с обеих сторон не добавляет дополнительной информации
    'hear_left', 'hear_right',  # Если слух с обеих сторон не добавляет дополнительной информации
    'serum_creatinine', 'SGOT_AST', 'SGOT_ALT', 'gamma_GTP'  # Если лабораторные тесты дублируют информацию
]

# Удаление столбцов
data = data.drop(columns=columns_to_drop, axis=1)

# Проверка результата
print(data.head())

   sex  age  height  weight  waistline    SBP   DBP   BLDS  tot_chole  \
0    1   35     170      75       90.0  120.0  80.0   99.0      193.0   
1    1   30     180      80       89.0  130.0  82.0  106.0      228.0   
2    1   40     165      75       91.0  120.0  70.0   98.0      136.0   
3    1   50     175      80       91.0  145.0  87.0   95.0      201.0   
4    1   50     165      60       80.0  138.0  82.0  101.0      199.0   

   HDL_chole  LDL_chole  triglyceride  hemoglobin  urine_protein  DRK_YN  
0       48.0      126.0          92.0        17.1            1.0       1  
1       55.0      148.0         121.0        15.8            1.0       0  
2       41.0       74.0         104.0        15.8            1.0       0  
3       76.0      104.0         106.0        17.6            1.0       0  
4       61.0      117.0         104.0        13.8            1.0       0  


In [9]:
# Проверка уникальных значений в целевой переменной
print("Уникальные значения в data['DRK_YN']:", data['DRK_YN'].unique())

Уникальные значения в data['DRK_YN']: [1 0]


In [11]:
# Разделение данных на признаки и целевую переменную
X = data.drop('DRK_YN', axis=1)
Y = data['DRK_YN']

In [13]:
# Масштабирование числовых признаков
scaler = StandardScaler()
numeric_features = X.select_dtypes(include=['int32', 'int64', 'float32', 'float64']).columns
X[numeric_features] = scaler.fit_transform(X[numeric_features])

In [15]:
# Разделение данных на обучающий и тестовый наборы
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

### Обучение модели методом решающего дерева


In [19]:
# Обучение модели методом решающего дерева
dt_classifier = DecisionTreeClassifier()

In [21]:
# Определяем сетку параметров для поиска оптимальных гиперпараметров модели дерева решений
dt_param_grid = {
    'criterion': ['gini', 'entropy'],             # Критерий измерения качества разделения
    'max_depth': [None, 10, 20, 30],              # Максимальная глубина дерева
    'min_samples_split': [2, 5, 10],              # Минимальное количество образцов для разделения узла
    'min_samples_leaf': [1, 2, 4]                 # Минимальное количество образцов для узла-листа
}

In [23]:
# Поиск оптимальных параметров с использованием кросс-валидации
dt_grid_search = GridSearchCV(dt_classifier, dt_param_grid, cv=5)
dt_grid_search.fit(X_train, Y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 10, 20, 30],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10]})

In [25]:
# Получение лучших параметров
best_dt_params = dt_grid_search.best_params_
print("Лучшие параметры для решающего дерева:", best_dt_params)

Лучшие параметры для решающего дерева: {'criterion': 'entropy', 'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 5}


In [27]:
# Обучение модели с лучшими параметрами
best_dt_classifier = DecisionTreeClassifier(**best_dt_params)
best_dt_classifier.fit(X_train, Y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=10, min_samples_leaf=2,
                       min_samples_split=5)

In [29]:
# Предсказание на тестовом наборе данных
Y_pred_dt = best_dt_classifier.predict(X_test)

In [31]:
# Оценка модели
print("Отчет о классификации для решающего дерева:")
print(classification_report(Y_test, Y_pred_dt))

Отчет о классификации для решающего дерева:
              precision    recall  f1-score   support

           0       0.70      0.64      0.67      1216
           1       0.66      0.71      0.68      1184

    accuracy                           0.68      2400
   macro avg       0.68      0.68      0.68      2400
weighted avg       0.68      0.68      0.68      2400



In [33]:
# Оценка модели на тестовом наборе данных
dt_accuracy = accuracy_score(Y_test, Y_pred_dt)
print("Точность решающего дерева на тестовом наборе данных:", dt_accuracy)

Точность решающего дерева на тестовом наборе данных: 0.6766666666666666


### Обучение модели методом случайного леса

In [37]:
# Обучение модели методом случайного леса
rf_classifier = RandomForestClassifier()

In [39]:
# Определяем сетку параметров для поиска оптимальных гиперпараметров модели случайного леса
rf_param_grid = {
    'n_estimators': [100, 200],             # Количество деревьев в лесу
    'criterion': ['gini', 'entropy'],            # Критерий измерения качества разделения
    'max_depth': [10, 20],             # Максимальная глубина деревьев
    'min_samples_split': [2, 5, 10],             # Минимальное количество образцов для разделения узла
    'min_samples_leaf': [1, 2, 4]                # Минимальное количество образцов для узла-листа
}

In [43]:
# Поиск оптимальных параметров с использованием кросс-валидации
rf_grid_search = GridSearchCV(rf_classifier, rf_param_grid, cv=2, n_jobs=-1, verbose=2)
rf_grid_search.fit(X_train, Y_train)

Fitting 2 folds for each of 72 candidates, totalling 144 fits


GridSearchCV(cv=2, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [10, 20], 'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200]},
             verbose=2)

In [45]:
# Получение лучших параметров
best_rf_params = rf_grid_search.best_params_
print("Лучшие параметры для случайного леса:", best_rf_params)

Лучшие параметры для случайного леса: {'criterion': 'entropy', 'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 200}


In [47]:
# Обучение модели с лучшими параметрами
best_rf_classifier = RandomForestClassifier(**best_rf_params)
best_rf_classifier.fit(X_train, Y_train)

RandomForestClassifier(criterion='entropy', max_depth=10, min_samples_leaf=4,
                       n_estimators=200)

In [49]:
# Предсказание на тестовом наборе данных
Y_pred_rf = best_rf_classifier.predict(X_test)

In [51]:
# Оценка модели
print("Отчет о классификации для случайного леса:")
print(classification_report(Y_test, Y_pred_rf))

Отчет о классификации для случайного леса:
              precision    recall  f1-score   support

           0       0.72      0.69      0.71      1216
           1       0.69      0.73      0.71      1184

    accuracy                           0.71      2400
   macro avg       0.71      0.71      0.71      2400
weighted avg       0.71      0.71      0.71      2400



In [53]:
# Оценка модели на тестовом наборе данных
rf_accuracy = accuracy_score(Y_test, Y_pred_rf)
print("Точность случайного леса на тестовом наборе данных:", rf_accuracy)

Точность случайного леса на тестовом наборе данных: 0.70875
